## 허깅페이스 튜토리얼
### 트랜스포머?
- 트랜스포머는 언어모델. 레이블 없이 자가 지도 학습 방식으로 학습된 모델.
- CNN, RNN처럼 인공지능 분야에서 많이 사용되는 인공신경망. chatGPT의 T가 트랜스포머.
- 허깅페이스는 트랜스포머를 기반으로 다양한 모델과 학습 스크립트를 구현해 놓은 데이터 사이언스 머신러닝 플랫폼.
- 트랜스포머 라이브러리는 모델을 사용하고 구축하는 기능을 제공한다.
- 트랜스포머는 실생활 문제를 해결하기 위해 사전 학습모델에게 지식을 전이받는 전이학습, 특정작업과 상황에 맞춘 미세조정이 필요하다.

### 간단하게 사용하는 방법
- 트랜스포머 모듈의 파이프라인 함수로 전처리 프롬프트를 보낸다.
- 모델로부터 후처리 결과를 받아서 출력해본다.
- 이외에 자연어처리 관련된 여러 기능을 사용할 수 있다.
  - 텍스트생성, 질의 응답, 감정 분석, 요약, 번역, 텍스트에 대한 특징 추출 등.
<br><br>
- https://huggingface.co/learn/nlp-course/ko/chapter1/3?fw=pt

In [3]:
# 설치 안내 페이지: https://huggingface.co/docs/transformers/installation
# !pip install transformers

In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis") # 문장의 감정을 파악해주는 디폴트 모델
res = classifier("I've been waiting for a HuggingFace course my whole life.")
print(res) # [{'label': 'POSITIVE', 'score': 0.9598045349121094}]
res2 = classifier("I hate you")
print(res2) # [{'label': 'NEGATIVE', 'score': 0.9991129040718079}]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


[{'label': 'POSITIVE', 'score': 0.9598045349121094}]
[{'label': 'NEGATIVE', 'score': 0.9991129040718079}]


### 어텐션 레이어
- Attention is All You Need. 트랜스포머는 특수한 레이어인 어텐션 레이어가 있다. 문장의 특정 단어에 주의를 기울이고 특정 단어는 무시하도록 알려준다.
- 구조(아키텍처) -> Bert, 체크포인트(가중치) -> bert-base-cased

### 트랜스포머 계열
- GPT계열 auto-regressive transformer, decoder models
  - 텍스트 생성에 특화
  - 문자 내 다음 단어 예측을 반복하는 방식으로 학습
  - 주어진 단어 앞에 위치한 단어에 액세스 <br><br>
- BERT계열 auto-encoding transformer
  - 문장에 대한 이해를 요구하는 작업에 특화
  - 랜덤으로 단어에 마스킹 하는 방식으로 학습해서 모델이 원본 문장을 찾아 재구성
  - 초기 문장의 모든 단어에 액세스 <br><br>
- BART/T5계열 sequence to sequence transformer, encoder-decoder models
  - 번역, 요약과 같이 입력을 필요로 하는 일에 특화
  - 인코더와 디코더 기능을 모두 사용하고 복잡함

### 편향과 한계
- 모델 리서처들이 무수히 많은 양의 데이터를 사전학습했기 때문에 양질의 데이터와 함께 그렇지 않은 데이터까지 수집해서 학습.
- 모델이 젠더, 인종, 동성애 등에 대한 혐오 표현할 가능성이 있다. 미세 조정을 거쳐도 내제된 편향성을 없애지 못한다.

In [2]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result]) # ['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])# ['nurse', 'maid', 'teacher', 'waitress', 'prostitute']

c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: 

['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


### 파이프라인 내부 동작 과정
- 전처리 작업 -> 모델 학습 -> 후처리 작업
1. 전처리:
     - 토큰화 -> 정수매핑 -> 부가적인 작업 -> 모델 학습
     - 모든 전처리 작업은 이용하는 모델의 사전 학습과정과 동일해야 하기 때문에 모델허브에서 다운 받은 AutoTokenizer의 from_pretrained()를 이용한다.
     - Transfrmer는 tensor만을 입력으로 받는다. tensor는 numpy 배열과 비슷하다. Numpy 배열은 스칼라(0D), 벡터(1D), 행렬(2D)과 같이 많은 차원을 가질 수 있다.
     <br><br>
     - https://huggingface.co/learn/nlp-course/ko/chapter2/2?fw=pt

In [1]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt") # pt = pytorch
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


2. 모델 학습
   - torch.Size([2, 16, 768])
   - batch size: 한 번에 처리되는 시퀀스 수 2
   - sequence length: 시퀀스 숫자 표현 길이 16
   - hidden size: 각 모델 입력 벡터 차원 768

In [3]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

outputs = model(**inputs) # 위에서 토큰화한 문장
print(outputs.last_hidden_state.shape) # torch.Size([2, 16, 768])

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([2, 16, 768])


3. 출력 후처리
   - 모델 헤드는 히든 스테이트의 고차원 벡터를 입력받아 다른 차원으로 투영.
   - 하나 또는 여러 개의 레이어로 이루어진 추가적인 요소로 트랜스포머의 예측 결과를 task-specific한 출력으로 변환합니다.
   -  헤드라고 알려진 적응 헤드는 언어 모델링 헤드, 질의 응답 헤드, 순차 분류 헤드 등과 같이 다양한 형태로 나타납니다.
   - 아래 로짓은 확률이 아니라 정규화 되지 않은 점수. 소프트맥스를 거쳐야 한다.
   - 소프트맥스는 상한과 하한을 적용해서 로짓 결과를 이해하기 쉽게 만든다.
<br><br>
4. 오토모델?
   - `from_pretrained`메소드를 통해서 적합한 아키텍처를 불러온다.
   - 사전학습이 된 웨이트, 설정, 단어 사전을 가져온다.

In [6]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

print(outputs.logits.shape) # torch.Size([2, 2])
print(outputs.logits)
# tensor([[-1.5607,  1.6123], -> 첫번째 문장 예측
#         [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>) -> 두번째 두장 예측



torch.Size([2, 2])
tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [7]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

# tensor([[4.0195e-02, 9.5980e-01],
#         [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [8]:
model.config.id2label # 모델의 레이블 얻기

{0: 'NEGATIVE', 1: 'POSITIVE'}

### 모델을 생성하고 사용하기


#### 모델의 아키텍처 직접 정의하는 방법
1. 무작위 값을 통한 기본 환경설정으로 모델 생성
    - 많은 시간, 많은 데이터가 필요하다.
    - 아래 컨피그 리스트는 학습을 위한 데이터

In [10]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



2. 사전학습된 모델 불러오기
     - BertConfig()를 사용하지 않고, bert-base-cased 식별자를 통해 사전 학습된 모델을 불러옴. BERT 개발자들이 직접 훈련시켰다.
     - 사전 훈련된 가중치를 직접 추론으로 바로 사용이 가능하거나,
     - 새로운 작업을 위해 미세 조정이 가능하다.
     - `~/.cache/huggingface/transformers`에서 캐시된 모델을 확인 할 수 있다. 한번 캐시된 모델을 또 호출해도 재다운로드하지 않는다.
     - BERT아키텍처와 관련된 많은 체크포인트가 모델 허브에 있다.

In [12]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")
sequences = ["Hello!", "Cool.", "Nice!"] # 시퀀스
encoded_sequences = [
    [101, 7592, 999, 102], # 인코딩된 시퀀스
    [101, 4658, 1012, 102], # input IDs
    [101, 3835, 999, 102], # 토크나이저로 토큰을 인덱스로 변환
]

import torch

model_inputs = torch.tensor(encoded_sequences) # transformer는 텐서만 입력 받음
output = model(model_inputs)
print(output)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 4.4496e-01,  4.8276e-01,  2.7797e-01,  ..., -5.4032e-02,
           3.9394e-01, -9.4770e-02],
         [ 2.4943e-01, -4.4093e-01,  8.1772e-01,  ..., -3.1917e-01,
           2.2992e-01, -4.1172e-02],
         [ 1.3668e-01,  2.2518e-01,  1.4502e-01,  ..., -4.6914e-02,
           2.8224e-01,  7.5566e-02],
         [ 1.1789e+00,  1.6738e-01, -1.8187e-01,  ...,  2.4672e-01,
           1.0441e+00, -6.1962e-03]],

        [[ 3.6436e-01,  3.2464e-02,  2.0258e-01,  ...,  6.0110e-02,
           3.2451e-01, -2.0996e-02],
         [ 7.1866e-01, -4.8725e-01,  5.1740e-01,  ..., -4.4012e-01,
           1.4553e-01, -3.7545e-02],
         [ 3.3223e-01, -2.3271e-01,  9.4876e-02,  ..., -2.5268e-01,
           3.2172e-01,  8.1106e-04],
         [ 1.2523e+00,  3.5754e-01, -5.1320e-02,  ..., -3.7840e-01,
           1.0526e+00, -5.6255e-01]],

        [[ 2.4042e-01,  1.4718e-01,  1.2110e-01,  ...,  7.6062e-02,
           3.3564e-01,  2

3. 저장방법
    - 2가지 파일을 저장함
    - ls directory_on_my_computer
    - config.json pytorch_model.bin
      - config.json은 모델 아키텍처를 구축하는데 필요한 속성을 알려준다.
      - transformer에 버전 등의 메타데이터를 포함하고 있다.

In [ ]:
model.save_pretrained("directory_on_my_computer")

### 토크나이저
- 토크나이저는 텍스트를 모델이 처리할 수 있는 데이터로 변환한다.
- 모델은 숫자만 처리할 수 있으니까 토크나이저는 텍스트 입력을 수치형 데이터로 변환한다.
- 서브워드 토큰화 알고리즘
  - 자주 사용되는 단어는 더 작은 서브워드로 나누면 안되지만, 희귀한 단어는 의미 있는 서브워드로 나눠야 한다는 규칙에 기반한다.
  - tokenization -> token + ization
  - 서브워드는 많은 양의 의미론적 정보를 제공
  - 긴 단어를 두 단어만으로 표현해서 공간 효율적
  - 크기가 작은 단어 사전으로 많은 토큰 표현
  - 언노운 토큰 거의 없다
- 과정
  1. 일정한 기준을 가지고 토큰 만들기
      -  단어 사전으로 표현할 수 있는 토큰을 얻을 때까지 단어를 분할한다.
      -  transformer -> trans + ##former
  2. 토큰을 인코딩해서 수치화하고 텐서에 담기
      - from_pretrained()메서드로 인스턴스화할 때 이미 다운받은 단어사전을 가지고 있고, 모델이 사용한 단어사전과 동일한 단어 사전을 사용해야 한다.
  <br><br>
- GPT - Byte-level BPE
- BERT - WordPiece
- etc - SentencePiece, Unigram

#### 인코딩

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [14]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


#### 디코딩
- 디코딩은 인코딩과 반대로 진행.
- 디코드 메서드는 인덱스를 토큰으로 바꾸기도 하지만, 읽기 좋은 문장을 만든다.

In [15]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple


#### 다중 시퀀스 처리


In [28]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

In [29]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [31]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [34]:
import numpy as np
np.reshape(tokens, (1,-1))

array([['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging',
        '##face', 'course', 'my', 'whole', 'life', '.']], dtype='<U7')

- 텐서로 만들기 전

In [41]:
 
print(np.reshape(ids,(1,-1)))
np.shape(ids)

[[ 1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026  2878
   2166  1012]]


(14,)

- 텐서로 만들었지만, 차원이 부족함

In [37]:
print(np.reshape(input_ids,(1,-1)))
np.shape(input_ids)

tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])


torch.Size([14])

- 어레이에 담아서 텐서로 만들기

In [39]:
input_ids = torch.tensor([ids])
np.shape(input_ids)

torch.Size([1, 14])

In [43]:
output = model(input_ids)
output.logits

tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)

- 동일한 길이의 다중 시퀀스는 문제가 되지 않음

In [45]:

batched_ids = [ids, ids]
input_bat = torch.tensor(batched_ids)
output = model(input_bat)
output.logits

tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)

- 길이가 다른 다중 시퀀스는 문제가 될 수 있다.
- 두 개 이상의 문장을 배치로 만드려고 할 때, 그 문장들은 서로 다른 길이를 가지고 있을 것이다.
- 문장 길이가 다르면 id 리스트를 텐서로 바로 변환할 수 없다. 그래서 패드를 추가한다.
- `tokenizer.pad_token_id`를 사용하여 패드를 넣는다.
- 하지만 서로 다른 토큰으로 인식이 된다.
- 트랜스포머 모델의 핵심 기능이 토큰을 문맥화하는 어텐션 레이어이기 때문이다.
- 어텐션 레이어에게 패드가 추가되었다는 사실을 알려줘야 한다. 그래서 패딩 토큰을 무시하라고 알려줘야 한다.

In [47]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


#### 어텐션 마스크
- 어텐션마스크가 패딩토큰의 유무를 가르쳐 준다.
- 1은 토큰을 주의깊게 봐달라는 뜻이고, 0은 무시해도 된다는 뜻이다.
- 입력 아이디 리스트와 같은 크기의 어텐션 마스크를 만들어 준다. 

In [48]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


#### 시퀀스의 길이가 길다면?
- 토큰의 최대길이는 모델마다 다르지만, 보통 512개나 1024개다.
- 시퀀스의 길이에 맞게 잘라야 한다.
- `sequence = sequence[:max_sequence_length]`

#### 한 번에 실행해보기

- Transformer API의 고수준 함수로 이 모든 과정을 빠르게 처리할 수 있다.
- 토크나이저의 __call__메서드는 거의 모든 것을 처리할 수 있는 강력한 메서드다.
- 모델로부터 예측 결과를 탐색하는 데에 사용이 된다.
- 토큰화, 입력 아이디로 변환, 패딩, 잘라내기, 어텐션 마스크 등.
- `AutoTokenizer.from_pretrained(checkpoint)`는 패딩+잘라내기+출력프레임워크를 포함해서 토큰화해준다.
- `__call__`과`convert_tokens_to_ids` 함수가 토큰을 아이디 리스트로 만들어준다.

In [53]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [55]:
tokenizer.tokenize("Hello")

['hello']

In [50]:
# 단일
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

In [ ]:
# 다중
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)

In [ ]:
# 가장 긴 시퀀스의 길이에 맞게 패딩을 추가합니다.
model_inputs = tokenizer(sequences, padding="longest")

# 모델이 지원하는 최대 시퀀스 길이에 맞게 패딩을 추가합니다.
# (BERT나 DistilBERT의 최대 길이는 512)
model_inputs = tokenizer(sequences, padding="max_length")

# 지정한 길이에 맞게 패딩을 추가합니다.
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# 모델이 지원하는 최대 시퀀스 길이에 맞게 시퀀스 길이를 잘라냅니다.
# (BERT나 DistilBERT의 최대 길이는 512)
model_inputs = tokenizer(sequences, truncation=True)

# 지정한 최대 길이에 맞게 시퀀스 길이를 잘라냅니다.
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# PyTorch 텐서를 반환합니다.
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# TensorFlow 텐서를 반환합니다.
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# NumPy 배열을 반환합니다.
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

#### 특수 토큰
- 토크나이저는 문장이 시작될 때 CLS, 끝날때 SEP이라는 토큰을 붙인다.
- 이런 토큰을 사용하는 이유는 모델이 사전학습될 때 이 토큰들을 사용했기 때문이고, 추론할 때 동일한 결과를 얻기 위함이다.
- 어떤 모델들은 특수 토큰을 사용하지 않고, 어떤 모델은 다른 토큰을 추가한다.

In [51]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [52]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


### 허깅페이스 허브 없이 데이터셋 이용하기
- 데이터셋을 로컬 또는 원격에서 로딩하는 방법
- 윈도우 wget, gzip 설치하기
  - wget: https://sound10000w.tistory.com/229
  - gzip: https://extrememanual.net/27173

#### 로컬에서 데이터셋 불러오기

In [5]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

--2023-07-14 17:12:03--  https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
Resolving github.com (github.com)... 20.200.245.247
Connecting to github.com (github.com)|20.200.245.247|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz [following]
--2023-07-14 17:12:03--  https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7725286 (7.4M) [application/octet-stream]
Saving to: 'SQuAD_it-train.json.gz'

     0K .......... .......... .......... .......... ..........  0% 16.0M 0s
    50K .......... .......... .......... .......... ..........  1% 47.5M 0s
   100K ......

In [7]:
!gzip -dkv SQuAD_it-*.json.gz

SQuAD_it-test.json.gz:	 87.5% -- replaced with SQuAD_it-test.json
SQuAD_it-train.json.gz:	 82.3% -- replaced with SQuAD_it-train.json


In [9]:
# !pip install datasets

  Using cached datasets-2.13.1-py3-none-any.whl (486 kB)
  Using cached pyarrow-12.0.1-cp39-cp39-win_amd64.whl (21.5 MB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     --------------------------------------- 10.8/10.8 MB 38.4 MB/s eta 0:00:00
  Using cached xxhash-3.2.0-cp39-cp39-win_amd64.whl (30 kB)
  Using cached multiprocess-0.70.14-py39-none-any.whl (132 kB)
  Using cached aiohttp-3.8.4-cp39-cp39-win_amd64.whl (323 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp39-cp39-win_amd64.whl (28 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached yarl-1.9.2-cp39-cp39-win_amd64.whl (61 kB)
  Using cached frozenlist-1.4.0-cp39-cp39-win_amd64.whl (44 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)


In [10]:
from datasets import load_dataset

squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/user/.cache/huggingface/datasets/json/default-7225ab3b28ff7f78/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['paragraphs', 'title'],
        num_rows: 442
    })
})

In [14]:
squad_it_dataset['train'][0]

{'paragraphs': [{'context': "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.",
   'qas': [{'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56cdca7862d2951400fa6826',
     'question': 'In quale anno si è verificato il terremoto nel Sichuan?'},
    {'answers': [{'answer_start': 232, 'text': '69.197'}],
     'id': '56cdca7862d2951400fa6828',
     'question': 'Quante persone sono state uccise come risultato?'},
    {'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56d4f9902ccc5a1400d833c0',
     'question': 'Quale anno ha avuto luogo il terremoto del Sichuan?'},
    {'answers': [{'answer_start': 78, 'text': '8.0 Ms e 7.9 Mw'}],
     'id': '56d4f9902ccc5a1400d833c1',
     'question': 'Che cosa ha fatto la misura di sisma?'},
    {'answers'

- 트레인과 테스트 데이터 둘 다 한번에 가져올 수 있다.

In [15]:
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/user/.cache/huggingface/datasets/json/default-45029a6e77bc5d00/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['paragraphs', 'title'],
        num_rows: 442
    })
    test: Dataset({
        features: ['paragraphs', 'title'],
        num_rows: 48
    })
})

- 사실 gzip 압축파일 자동해제를 지원한다.

In [ ]:
data_files = {"train": "SQuAD_it-train.json.gz", "test": "SQuAD_it-test.json.gz"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

#### 원격 데이터셋 로딩해보기

In [ ]:
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

### 커뮤니티와 에러 대응

- 주피터 노트북에서 허깅페이스 허브 로그인

In [16]:
# !pip install huggingface_hub

- 반드시 액세스토큰은 Write용으로 발급을 받고 로그인 토큰으로 사용하자.

In [5]:
from huggingface_hub import notebook_login

notebook_login()

# huggingface-cli login # 터미널용

In [2]:
from distutils.dir_util import copy_tree
from huggingface_hub import Repository, snapshot_download, create_repo, get_full_repo_name


def copy_repository_template():
    # Clone the repo and extract the local path
    template_repo_id = "lewtun/distilbert-base-uncased-finetuned-squad-d5716d28"
    commit_hash = "be3eaffc28669d7932492681cd5f3e8905e358b4"
    template_repo_dir = snapshot_download(template_repo_id, revision=commit_hash)
    # Create an empty repo on the Hub
    model_name = template_repo_id.split("/")[1]
    create_repo(model_name, exist_ok=True)
    # Clone the empty repo
    new_repo_id = get_full_repo_name(model_name)
    new_repo_dir = model_name
    repo = Repository(local_dir=new_repo_dir, clone_from=new_repo_id)
    # Copy files
    copy_tree(template_repo_dir, new_repo_dir)
    # Push to Hub
    repo.push_to_hub()

In [3]:
copy_repository_template()

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Cloning https://huggingface.co/lake-crimsonn/distilbert-base-uncased-finetuned-squad-d5716d28 into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/2.05k [00:00<?, ?B/s]

To https://huggingface.co/lake-crimsonn/distilbert-base-uncased-finetuned-squad-d5716d28
   e72f28a..7b36ee2  main -> main



- 내 레포지토리에 올라온 모델 확인하기

In [6]:
from transformers import pipeline
from huggingface_hub import Repository, snapshot_download, create_repo, get_full_repo_name

model_checkpoint = get_full_repo_name("distilbert-base-uncased-finetuned-squad-d5716d28")
reader = pipeline("question-answering", model=model_checkpoint)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5a43ae3c-889d-4380-af4a-70892eccdad9)')' thrown while requesting HEAD https://huggingface.co/lake-crimsonn/distilbert-base-uncased-finetuned-squad-d5716d28/resolve/main/config.json


- `list_repo_files`모델을 빠르게 액세스하는 방법이면서, 허브에 올라온 파일들을 확인할 수 있다.
- config.json이 없으면 파이프라인이 이 모델을 불러올 수 없다.

In [7]:
from huggingface_hub import list_repo_files

list_repo_files(repo_id=model_checkpoint)

['.gitattributes',
 'README.md',
 'config.json',
 'pytorch_model.bin',
 'special_tokens_map.json',
 'tokenizer_config.json',
 'training_args.bin',
 'vocab.txt']

- 사전 훈련이 된 모델이 `distilbert-base-uncased`로 확인이 되었다, 디폴트컨피그를 받아서 컨피그를 업데이트 해주겠다.

In [8]:
from transformers import AutoConfig

pretrained_checkpoint = "distilbert-base-uncased"
config = AutoConfig.from_pretrained(pretrained_checkpoint)

In [38]:
config.push_to_hub(model_checkpoint, commit_message="Add config.json")

CommitInfo(commit_url='https://huggingface.co/lake-crimsonn/distilbert-base-uncased-finetuned-squad-d5716d28/commit/d5c4a730d9e80555670868a93202e9e2d27f2efb', commit_message='Add config.json', commit_description='', oid='d5c4a730d9e80555670868a93202e9e2d27f2efb', pr_url=None, pr_revision=None, pr_num=None)

- 메인브랜치의 최근 커밋에서 모델을 로드할 수 있다.

In [9]:
reader = pipeline("question-answering", model=model_checkpoint, revision="main")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text
given a question. An example of a question answering dataset is the SQuAD
dataset, which is entirely based on that task. If you would like to fine-tune a
model on a SQuAD task, you may leverage the
examples/pytorch/question-answering/run_squad.py script.

🤗 Transformers is interoperable with the PyTorch, TensorFlow, and JAX
frameworks, so you can use your favourite tools for a wide variety of tasks!
"""

question = "What is extractive question answering?"
reader(question=question, context=context)

{'score': 0.3761069178581238,
 'start': 34,
 'end': 95,
 'answer': 'the task of extracting an answer from a text\ngiven a question'}

---